# ML TOOL 

<span> A jupyter notebook tool to aid the testing of machine learning (ML) models in the domain of activity detection.  </span>

### US-01
Ensure jupyter notebook has python3.x 

In [1]:
from platform import python_version

print(python_version())

3.8.10


### Installing Dependencies


In [2]:
%pip install torch==1.10.1 torchvision==0.11.2 torchaudio==0.10.1 -f https://download.pytorch.org/ehl/torch_stable.html
%pip install timm==0.4.12
%pip install scikit-learn
%pip install numpy

Looking in links: https://download.pytorch.org/ehl/torch_stable.html
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Installation of required Libraries 
!pip install ipywidgets
!pip install jupyter-js-widgets-nbextension
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [6]:
!pip install ipyfilechooser

### Dependencies and Setup
<font color='red'>All imports put here.</font>

In [7]:
import torch
import os
import ipywidgets as widgets
from subprocess import Popen
print(torch.__version__)
from pathlib import Path

1.10.1+cu102


### US-03
Data folder that stores different subfolders to store different types of datasets accordingly

In [8]:
# Create data folder if it does not exist 

data_directory = Path('./data')


if not os.path.exists(data_directory):
    os.makedirs(data_directory)

In [9]:
# Get user to input the number of folders needed 
print("How many datasets would you like to upload?")
inputbounded = widgets.BoundedIntText(
    value=1,
    min=0,
    max=10,
    step=1,
    description='Datasets:',
    disabled=False
)
inputbounded

How many datasets would you like to upload?


BoundedIntText(value=1, description='Datasets:', max=10)

In [10]:
# Takes in user input folder name for datasets to be created in "./data/_____"
numdatasets = inputbounded.value
count = 0

while count < numdatasets:
    foldername = input("Enter Dataset Name: ")
    
    directory = data_directory/foldername
    if not os.path.exists(directory):
        os.makedirs(directory)
        count += 1
        print("Folder " +foldername + " is created.")
        print("Please upload video data to this file path: "+ str(directory))
    else:
        print("Invalid Folder Name!")

KeyboardInterrupt: Interrupted by user

### US-04 
Select Video to playback

In [ ]:
# Select a folder from the datasets folder created
list_files = os.listdir(data_directory)
print("Please choose a dataset")
folderbounded = widgets.Dropdown(
    options=list_files,
    description = 'Dataset:',
    disabled=False
)
folderbounded

In [ ]:
folder_name = data_directory/folderbounded.value

# Select a folder from the datasets folder created
list_files = os.listdir(folder_name)
        
print("Please select a video to playback.")      
filebounded = widgets.Dropdown(options=list_files,
                               description = str(folder_name)+':',
                               disabled=False)
filebounded

In [ ]:
from IPython.display import Video

Video(folder_name/filebounded.value, embed=True) 

### US-06
Loading a model


In [ ]:
mylist = os.listdir("trained_model")
selectedModel = widgets.Dropdown(
    options=mylist,
    value = mylist[0],
    description = 'Choose Model',
    disabled=False
)
selectedModel

In [ ]:
selectedModel.value

### US-07
Choose dataset from TSU project

In [ ]:
selectedFeatureData = widgets.Dropdown(
    options=["i3d", "3DPose"],
    value = "i3d",
    description = 'Choose Feature',
    disabled=False
)

selectedFeatureData

In [ ]:
if (selectedFeatureData.value == "i3d"):
    dataPath = "./Toyota_Smarthome/pipline/data/TSU_RGB_i3d_feat/RGB_i3d_16frames_64000_SSD/"
else:
    dataPath = "./Toyota_Smarthome/pipline/data/TSU_3DPose_AGCN_feat/2sAGCN_16frames_64000/"

dataPath

In [ ]:
!chmod 755 -R ./Toyota_Smarthome/pipline/run_PDAN.sh

#linux
Process=Popen("./Toyota_Smarthome/pipline/run_PDAN.sh %s %s" % (str(selectedModel.value), str(dataPath)), shell=True)
#!./Toyota_Smarthome/pipline/run_PDAN.sh selectedModel

### US-09
Choose a dataset subfolder from the data folder to use for training

In [ ]:
from ipyfilechooser import FileChooser

starting_directory = data_directory
folderupload = FileChooser(starting_directory)
# Switch to folder-only mode
folderupload.show_only_dirs = True
folderupload.title = '<b>Folder Upload<b>'
display(folderupload)

In [ ]:
print(folderupload.selected)
print(folderupload.selected_path)

### US-11
Specify a name for the new model

In [15]:
trainedmodeltxtval = widgets.Text(
    value='',
    placeholder='Name for newly trained model?',
    description='Input Name:',
    disabled=False
)

trainedmodeltxtval

Text(value='', description='Input Name:', placeholder='Name for newly trained model?')

In [16]:
print(trainedmodeltxtval.value)

testestest


### US-16
Choose a dataset subfolder from the data folder to use for testing

In [13]:
# Files / folders under data folder
datalist = os.listdir('./Toyota_Smarthome/pipline/data/')
widgets.Box(
    [
        widgets.Label(value='Choose a dataset:'),
        widgets.RadioButtons(
            options=datalist,
            layout={'width': 'max-content'}
        )
    ]
)

Box(children=(Label(value='Choose a dataset:'), RadioButtons(layout=Layout(width='max-content'), options=('sma…